## Trading Bot - ML Strategy

In [20]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import fxcmpy
import time
from datetime import datetime
import pickle

In [21]:
FXCM_API_TOKEN = "990af1044016076937a455014dd0a28921b0142e"
LOG_FILE = "fxcm.log"
LOG_LEVEL = "error"
SERVER = "demo"

In [22]:
api = fxcmpy.fxcmpy(access_token=FXCM_API_TOKEN, log_file=LOG_FILE, log_level=LOG_LEVEL, server=SERVER)

ServerError: Can not connect to FXCM Server.

In [ ]:
col = ["tradeId", "amountK", "currency", "grossPL", "isBuy"]

In [ ]:
class MLTrader():

  def __init__(self, instrument, bar_length, lags, model, units):
    self.instrument = instrument
    self.bar_length = pd.to_timedelta(bar_length)
    self.tick_data = None
    self.raw_data = None
    self.data = None
    self.ticks = 0
    self.last_bar = None
    self.units = units
    self.position = 0

    self.lags = lags
    self.model = model

  def get_most_recent(self, period="m1", number = 10000):

    while True:
      time.sleep(5)
      df = api.get_candles(self.instrument, number = number, period = period, columns = ['bidclose', 'askclose'])
      df['self.instrument'] = (df.bidclose + df.askclose) / 2
      df = df[self.instrument].to_frame()
      df = df.resample(self.bar_length, label = 'right').last().dropna().iloc[:-1]
      self.raw_data = df.copy()
      self.last_bar = self.raw_data.index[-1]
      if pd.to_datetime(datetime.utcnow()) - self.last_bar < self.bar_length:
        break

  def get_tick_data(self, data, dataframe):

    self.ticks += 1
    print(self.ticks, end=" ")

    recent_tick = pd.to_datetime(data['Updated'], unit = 'ms')

    if recent_tick - self.last_bar > self.bar_length:

      self.tick_data = dataframe.loc[self.last_bar:, ["Bid", "Ask"]]
      self.tick_data[self.instrument] = (self.tick_data.Ask + self.tick_data.Bid) / 2
      self.tick_data = self.tick_data[self.instrument].to_frame()
      self.resample_and_join()
      self.define_strategy()
      self.execute_trades()

  def resample_and_join(self):
    self.raw_data = self.raw_data.append(self.tick_data.resample(self.bar_length, label='right').last().ffill().iloc[:-1])
    self.last_bar = self.raw_data.index[-1]

  def define_strategy(self):

    df = self.raw_data.copy()

    df = df.appends(elf.tick_data.iloc[-1])
    data['returns'] = np.log(df[self.instrument] / df[self.instrument].shift())

    cols = []

    for lag in range(1, self.lags + 1):

      col = f"lag{lag}"
      df[col] = df.returns.shift(lag)
      cols.append(col)

    df.dropna(inplace=True)
    df["position"] = lr.predict(df[cols])

    self.data = df.copy()

  def execute_trades(self):

    if self.data['position'].iloc[-1] == 1:
      if self.position == 0:
        order = api.create_market_buy_order(self.instrument, self.units)
        self.report_trade(order, "GOING LONG")

      elif self.position == -1:
        order = api.create_market_buy_order(self.instrument, self.units * 2)
        self.report_trade(order, "GOING LONG")

      self.position = 1
    
    elif self.data['position'].iloc[-1] == -1:
      if self.position == 0:
        order = api.create_market_sell_order(self.instrument, self.units)
        self.report_trade(order, "GOING SHORT")

      elif self.position == 1:
        order = api.create_market_sell_order(self.instrument, self.units * 2)
        self.report_trade(order, "GOING SHORT")
      
      self.position = -1

    elif self.data["position"].loc[-1] == 0:
      if self.position == -1:
        order = api.create_market_buy_order(self.instrument, self.units)
        self.report_trade(order, "GOING NEUTRAL")

      elif self.position == 1:
        order = api.create_market_sell_order(self.instrument, self.units)
        self.report_trade(order, "GOING NEUTRAL")

      self.position = 0


  def report_trade(self, order, going):
    time = order.get_time()
    units = api.get_open_positions().amountK.iloc[-1]
    price = api.get_open_positions().open.iloc[-1]
    unreal_pl = api.get_open_positions().grossPl.sum()
    print("\n" + 100* "-")
    print(f"{time} | {going}")
    print(f"{time} | units = {units} | price = {price} | Unreal. P&L = {unreal_pl}")
    print(100* "-" + "\n")


In [ ]:
lr = pickle.load(open('linear_reg.p', 'rb'))

In [ ]:
trader = MLTrader(instrument="EUR/USD", bar_length="5min", lags=2, model=lr, units=100)

In [ ]:
trader.model

In [ ]:
trader.get_most_recent()
api.subscribe_market_data(trader.instrument, (trader.get_tick_data,))

In [ ]:
api.unsubscribe_market_data(trader.instrument)
if len(api.get_open_positions()) != 0:
  api.close_all_for_symbol(trader.instrument)
  print(2*f"\n"+"{str(datatime.utcnow())} | GOING NEUTRAL")
  time.sleep(20)
  print(api.get_closed_positions_summary()[col])
  trader.position = 0

In [ ]:
trader.data.tail(10)

In [ ]:
api.close()